[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/RajGajjar-01/LangChain-LangGraph/blob/main/Langgraph/15_Tool_calling.ipynb)

In [ ]:
# Install dependencies if running in Google Colab
import sys
if 'google.colab' in sys.modules:
    !pip install -U langgraph langchain-google-genai langchain-huggingface python-dotenv

## **Tool Calling in LangGraph**

<div class="premium-card">
    This notebook demonstrates how to implement a basic <b>tool-calling pattern</b> using LangGraph. We create a set of simple mathematical tools and a stateful graph that can route messages between a chatbot and these tools.
</div>

### **1. Environment Setup and Imports**

First, we load environmental variables and import necessary components from LangChain and LangGraph.

In [ ]:
from dotenv import load_dotenv
from typing import Annotated, TypedDict

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition

In [ ]:
load_dotenv()

### **2. Define Tools**

We define basic arithmetic operations as tools using the `@tool` decorator. These tools will be called by the LLM when needed.

In [ ]:
@tool
def add(a:int, b:int) -> int:
    """Add two numbers"""
    return a + b

@tool
def multiply(a:int, b:int) -> int:
    """Multiply two numbers"""
    return a * b

@tool
def subtract(a:int, b:int) -> int:
    """Subtract two numbers"""
    return a - b

@tool
def divide(a:int, b:int) -> int:
    """divide two numbers"""
    return a // b

tools = [add, multiply, subtract, divide]

### **3. Define Graph State**

The state of our graph is a simple dictionary containing a list of messages. We use `Annotated` with `add_messages` to ensure that new messages are <b>appended</b> to the existing list.

In [ ]:
class State(TypedDict):
    messages : Annotated[list, add_messages]

### **4. Define Nodes (Chatbot)**

We define nodes for our graph. The <code>chatbot</code> node uses a model (HuggingFace or Google Gemini) bound with our tools.

In [ ]:
def chatbot2(state: State) -> State:
    llm = ChatGoogleGenerativeAI(
        model="gemini-2.5-flash",
    )

    llm_with_tools = llm.bind_tools(tools)
    response = llm_with_tools.invoke(state['messages'])
    return {"messages": [response]}

In [ ]:
def chatbot(state: State) -> State:
    model = HuggingFaceEndpoint(
        repo_id='meta-llama/Llama-3.3-70B-Instruct',
        task="text-generation",
        max_new_tokens=2048,
    )

    llm = ChatHuggingFace(llm = model)

    llm_with_tools = llm.bind_tools(tools)
    response = llm_with_tools.invoke(state['messages'])
    return {"messages": [response]}

### **5. Build the Graph**

Now we assemble the components into a <code>StateGraph</code>. We add nodes for the chatbot and the tool executor, and define the edges including conditional edges to handle tool calls.

In [ ]:
graph = StateGraph(State)

graph.add_node("chatbot", chatbot)

tool_node = ToolNode(tools=tools)

graph.add_node("tools", tool_node)

graph.add_edge(START, "chatbot")

graph.add_conditional_edges("chatbot", tools_condition)

graph.add_edge("tools", "chatbot")

workflow = graph.compile()

### **6. Visualize the Graph**

We can visualize the graph structure using Mermaid. This helps in understanding the control flow.

In [ ]:
from IPython.display import Image
Image(workflow.get_graph().draw_mermaid_png())

### **7. Run the Workflow**

Finally, we run the workflow with a mathematical question to see the <b>tool calling</b> in action.

In [ ]:
question = "Compute (20 + 10) * 4 - 4 / 2. Show steps and use podmas"

result = workflow.invoke(
    {"messages": [HumanMessage(content=question)]}
)

print("\n===== FINAL ANSWER =====\n")
print(result["messages"][-1].content)

print("\n===== TRACE (messages) =====\n")
for m in result["messages"]:
    print(type(m).__name__, "=>", getattr(m, "content", ""))